In [6]:
from gurobipy import *
import numpy as np
import pandas as pd
from pandas import DataFrame
import csv

In [8]:
try:

    #Create a model
    m = Model("Project")

     
    #Data (Provided)
    utc_values = np.array(pd.read_csv('~/NetworkDesign/dc60_cust6000_inst1/unit_tran_cost.txt', header=None))
    dc_cost_values = np.array(pd.read_csv('~/NetworkDesign/dc60_cust6000_inst1/dc_cost.txt', header=None))
    dc_cap_values = np.array(pd.read_csv('~/NetworkDesign/dc60_cust6000_inst1/dc_cap.txt', header=None))
    demand_values = np.array(pd.read_csv('~/NetworkDesign/dc60_cust6000_inst1/demand.txt', header=None))
    
    #Initialize values
    x = utc_values.shape
    numCustomers=x[0]
    numDCs = x[1]
    
    #Create variables
    utc_vars = []    #unit transport cost variables
    dc_cost_vars = []     #distribution centers fixed cost variables
    dc_cap_vars = []
    
    for i in range(1,numCustomers+1):
        
        for j in range(1,numDCs+1):
            v = "x"+str(i)+"_"+str(j)
            v = m.addVar(vtype=GRB.INTEGER, name=v)
            utc_vars.append(v)
            
            if(i==1):
                p = "y"+str(j)
                p = m.addVar(vtype=GRB.BINARY, name=p)
                dc_cost_vars.append(p)
    #Update variables
    m.update()
    
    #Shape arrays to match data shape
    dc_cost_vars = np.array(dc_cost_vars)
    dc_cost_vars = dc_cost_vars.reshape(numDCs,1)
    utc_vars = np.array(utc_vars)
    utc_vars = utc_vars.reshape(numCustomers,numDCs)
    
     
    #Objective Function
    m.setObjective(np.sum(utc_vars*utc_values)+np.sum(dc_cost_vars*dc_cost_values), GRB.MINIMIZE)
    
    #Constraints
    #Customer Demand
    i=-1
    j=-1
    for v in demand_values:
        i+=1
        m.addConstr(np.sum(utc_vars[i])==demand_values[i],"DemandConstr"+str(i+1))
    
    #Distribution Center Capacity Constraints
    for c in dc_cap_values:
        j+=1
        m.addConstr(np.sum(utc_vars[:,j])<=dc_cap_values[j], "CapacityConstr"+str(j+1))
    
    #Optimize model
#    m.optimize()
 
#     for val in m.getVars():
#         if val.x != 0:
#             print val.varName, val.x
        
    print "Objective Value: ", m.objVal
    
    
except GurobiError:
    print 'Error reported. Solution not found'

Optimize a model with 5050 rows, 250050 columns and 500000 nonzeros
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [1e+00, 9e+04]
  Bounds range    [1e+00, 1e+00]
  RHS range       [2e+01, 3e+03]
Found heuristic solution: objective 8.05875e+06
Presolve removed 0 rows and 50 columns
Presolve time: 0.64s
Presolved: 5050 rows, 250000 columns, 500000 nonzeros
Variable types: 0 continuous, 250000 integer (0 binary)

Root relaxation: objective 1.669969e+06, 9226 iterations, 2.19 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    1669969.0000 1669969.00  0.00%     -    2s

Explored 0 nodes (9226 simplex iterations) in 2.98 seconds
Thread count was 4 (of 4 available processors)

Optimal solution found (tolerance 1.00e-04)
Best objective 1.669969000000e+06, best bound 1.669969000000e+06, gap 0.0%
Objective Value:  1669969.0


/home/akinfemi/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:53: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


In [30]:
res_df


,Variables,Values


In [27]:
res_df.head()

,Variables,Values


In [24]:
dc_cap_values.shape

(20, 1)

In [8]:
res = DataFrame(columns=["Variable", "Values"])

In [11]:
res.loc[0] = ["z",1]

In [24]:
res.append(DataFrame([['y',2]],columns=['Variable','Values']),ignore_index=True)

,Variable,Values
0,z,1
1,y,2


In [21]:
DataFrame([["y",2]])

,0,1
0,y,2
